In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np
import pyautogui
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume


def set_volume(volume, increment):
    # Get speaker interface and volume range
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume.iid, CLSCTX_ALL, None)
    volume_interface = cast(interface, POINTER(IAudioEndpointVolume))
    vol_min, vol_max = volume_interface.GetVolumeRange()[:2]

    # Get current volume and calculate new volume
    current_vol = volume_interface.GetMasterVolumeLevel()
    new_vol = current_vol + increment

    # Check if new volume is within range and set volume
    if new_vol < vol_min:
        volume_interface.SetMasterVolumeLevel(vol_min, None)
    elif new_vol > vol_max:
        volume_interface.SetMasterVolumeLevel(vol_max, None)
    else:
        volume_interface.SetMasterVolumeLevel(new_vol, None)

    # Calculate new volume percentage
    vol_percent = int((new_vol - vol_min) / (vol_max - vol_min) * 100)

    return vol_percent

mpHands = mp.solutions.hands # detects hand/finger
hands = mpHands.Hands()   # complete the initialization configuration of hands

# Example usage of set_volume function
# Assuming current volume is 50%
current_volume = 0.5  # Range: 0.0 - 1.0
increment = 0.1  # Increase volume by 10%

# Convert current volume to volume level
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume.iid, CLSCTX_ALL, None)
volume_interface = cast(interface, POINTER(IAudioEndpointVolume))
vol_min, vol_max = volume_interface.GetVolumeRange()[:2]
current_vol_level = int(current_volume * (vol_max - vol_min) + vol_min)

# Call set_volume function
new_vol_percent = set_volume(current_vol_level, int(increment * (vol_max - vol_min)))

def calculate_reference_distance():
    cap = cv2.VideoCapture(0) #Checks for camera
    reference_lmList = []
    while len(reference_lmList) == 0:
        success,img = cap.read() #If camera works capture an image
        imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb

        # Collection of gesture information
        results = hands.process(imgRGB) # completes the image processing.

        lmList = [] # empty list
        if results.multi_hand_landmarks: # list of all hands detected.
            # By accessing the list, we can get the information of each hand's corresponding flag bit
            for handlandmark in results.multi_hand_landmarks:
                for id,lm in enumerate(handlandmark.landmark): # adding counter and returning it
                    # Get finger joint points
                    h,w,_ = img.shape
                    cx,cy = int(lm.x*w),int(lm.y*h)
                    lmList.append([id,cx,cy]) # adding to the empty list 'lmList'
                mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)

        if lmList != []:
            # getting the value at a point
                            # x      # y
            x1,y1 = lmList[4][1],lmList[4][2]  # thumb
            x2,y2 = lmList[8][1],lmList[8][2]  # index finger
            # creating circle at the tips of thumb and index finger
            cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) # image # fingers # radius # rgb
            cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) # image # fingers # radius # rgb
            cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  # create a line b/w tips of index finger and thumb

            reference_length = hypot(x2-x1,y2-y1) # distance b/w tips using hypotenuse
            reference_lmList = lmList

        cv2.imshow('Image',img) # Show the video 
        if cv2.waitKey(1) & 0xff==ord(' '): # By using spacebar delay will stop
            break

    cap.release()     # stop cam       
    cv2.destroyAllWindows() # close window
    
    return reference_length, reference_lmList

# Set up the hand tracking model
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                        max_num_hands=1,
                        min_detection_confidence=0.7,
                        min_tracking_confidence=0.5)

# Set up the screen size
screen_width, screen_height = pyautogui.size()

# Set the volume range (0 to 100)
volume_range = 100

# Start the camera
cap = cv2.VideoCapture(0)

# Initialize the hand landmarks
lm_list = []

# Loop through the video frames
while True:
    # Capture the video frame
    ret, frame = cap.read()

    # Flip the frame horizontally for a mirrored view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB for Mediapipe processing
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe
    results = hands.process(frame_rgb)

    # Draw the hand landmarks on the frame
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the coordinates of the thumb tip, index finger tip, wrist, and middle finger tip
            thumb_tip = hand_landmarks.landmark[4]
            index_tip = hand_landmarks.landmark[8]
            wrist = hand_landmarks.landmark[0]
            middle_tip = hand_landmarks.landmark[12]
            
            thumb_x, thumb_y = thumb_tip.x * screen_width, thumb_tip.y * screen_height
            index_x, index_y = index_tip.x * screen_width, index_tip.y * screen_height
            wrist_x, wrist_y = wrist.x * screen_width, wrist.y * screen_height
            middle_x, middle_y = middle_tip.x * screen_width, middle_tip.y * screen_height

            # Calculate the distance between the thumb tip and index finger tip
            distance = math.sqrt((thumb_x - index_x) ** 2 + (thumb_y - index_y) ** 2)

            # Calculate the normalization factor (distance between wrist and middle finger tip)
            norm_factor = math.sqrt((wrist_x - middle_x) ** 2 + (wrist_y - middle_y) ** 2)

            # Normalize the distance
            normalized_distance = distance / norm_factor

            # Map the normalized distance to the volume range
            volume = int(np.interp(normalized_distance, [0, 1], [0, volume_range]))

            # Set the system volume
            pyautogui.press('volumedown') if volume < 50 else pyautogui.press('volumeup')
            pyautogui.press('volumemute') if volume == 0 else None

            # Print the volume on the frame
            cv2.putText(frame, f'Volume: {volume}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow('Frame', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

109


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
